![money](img/money_edit4.png)

<center> <h1> Where is my money? </h1> </center>
<center> <h2> Data Science as a tool of citizenship.</h2> </center>

## Introduction:  <a name="intro"></a> <br />

<div style="text-align: justify"> The availability of public data allows citizens to investigate the quality of public administration. Let's take a look on data made available by brazilian federal government. However, it is increasingly common to access data provided by governments from around the free world.
</div>

**About dataset:** <br />
<div style="text-align: justify"> The <a href="https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps">dataset</a> can be downloaded from the website of the Brazilian federal senate. These values are called Quotas for the Exercise of Parliamentary Activity for Senators in Portuguese CEAPs. It is made available by the exercise of the mandate of the senators.

**Motivation:** <br />
Brazilian senators have available amounts for spending on parliamentary activity. Let's look at how public money is being spent And answer the following questions: <br >
What is the change in total spending in recent years? <br >
Which senators spent the most in the year under review?<br >
Which party has the most spending?

## Preparing the data: <br/>

In [2]:
## Imports
import pandas as pd
import numpy as np
from matplotlib.ticker import MaxNLocator

### Plot
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
## Configuration
%matplotlib inline
#plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = 15,5

In [6]:
## Data load
path = "data/"

# Ano atual de analise
df_2018 = pd.read_csv(path+"2018.csv", skiprows=1, encoding='latin1', delimiter=';')

# Anos anteriores
df_2017 = pd.read_csv(path+"2017.csv", skiprows=1, encoding='latin1', delimiter=';')
df_2016 = pd.read_csv(path+"2016.csv", skiprows=1, encoding='latin1', delimiter=';')
df_2015 = pd.read_csv(path+"2015.csv", skiprows=1, encoding='latin1', delimiter=';')
df_2014 = pd.read_csv(path+"2014.csv", skiprows=1, encoding='latin1', delimiter=';')

# Mapa de nomes dos senadores com partidos
siglas = pd.read_csv(path+"names.csv")

## Exploratory analysis - 2018 <br />
<p style='text-align: justify;'> A basic analysis will be done in the year 2018. We did not analyze the year 2019 because it is the current year and the data will change.

In [7]:
df_2018.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
0,2018,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,02/01/2018,Despesa com pagamento de aluguel de imóvel par...,6000
1,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,02/01/2018,"Companhia Aérea: AVIANCA, Localizador: M2CJMM....","274,99"
2,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,02/01/2018,"Companhia Aérea: LATAM, Localizador: SFEXYF. P...","1094,29"
3,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,24/01/2018,"Companhia Aérea: AZUL, Localizador: V64HTR. Pa...","1553,99"
4,2018,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,002/18,01/02/2018,Despesa com pagamento de aluguel de imóvel par...,6000


In [8]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24492 entries, 0 to 24491
Data columns (total 10 columns):
ANO                  24492 non-null int64
MES                  24492 non-null int64
SENADOR              24492 non-null object
TIPO_DESPESA         24492 non-null object
CNPJ_CPF             24492 non-null object
FORNECEDOR           24492 non-null object
DOCUMENTO            23476 non-null object
DATA                 24492 non-null object
DETALHAMENTO         18882 non-null object
VALOR_REEMBOLSADO    24492 non-null object
dtypes: int64(2), object(8)
memory usage: 1.9+ MB


## Data Formatting

In the end we will have a function that helps us in the treatment of data from previous and next years.

In [9]:
# Transformar em valores númericos
df_2018["VALOR_REEMBOLSADO"] = df_2018["VALOR_REEMBOLSADO"].str.replace(",",".")
df_2018['VALOR_REEMBOLSADO'] = pd.to_numeric(df_2018['VALOR_REEMBOLSADO'])

In [10]:
# Drop data pois tenho mês e anod
df_2018 = df_2018.drop("DATA", axis=1)

Treatment of column "Tipo de despesas"

In [11]:
df_2018["TIPO_DESPESA"].unique()

array(['Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.',
       'Passagens aéreas, aquáticas e terrestres nacionais',
       'Divulgação da atividade parlamentar',
       'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar',
       'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes',
       'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ',
       'Serviços de Segurança Privada'], dtype=object)

In [12]:
format_values = {'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.': 'Rent',
 'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ': 'General expenses',
 'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar': 'Hiring',
 'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes': 'Daily Spend',
 'Passagens aéreas, aquáticas e terrestres nacionais': 'Airline tickets',
 'Divulgação da atividade parlamentar': 'Advertising',
 'Serviços de Segurança Privada': 'Security'}

In [13]:
df_2018['TIPO_DESPESA'] = df_2018['TIPO_DESPESA'].map(format_values)

## Take a look after treatment:

In [14]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24492 entries, 0 to 24491
Data columns (total 9 columns):
ANO                  24492 non-null int64
MES                  24492 non-null int64
SENADOR              24492 non-null object
TIPO_DESPESA         24492 non-null object
CNPJ_CPF             24492 non-null object
FORNECEDOR           24492 non-null object
DOCUMENTO            23476 non-null object
DETALHAMENTO         18882 non-null object
VALOR_REEMBOLSADO    24492 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.7+ MB


In [15]:
df_2018.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DETALHAMENTO,VALOR_REEMBOLSADO
0,2018,1,ACIR GURGACZ,Rent,004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,Despesa com pagamento de aluguel de imóvel par...,6000.00
1,2018,1,ACIR GURGACZ,Airline tickets,63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,"Companhia Aérea: AVIANCA, Localizador: M2CJMM....",274.99
2,2018,1,ACIR GURGACZ,Airline tickets,63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,"Companhia Aérea: LATAM, Localizador: SFEXYF. P...",1094.29
3,2018,1,ACIR GURGACZ,Airline tickets,63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,"Companhia Aérea: AZUL, Localizador: V64HTR. Pa...",1553.99
4,2018,2,ACIR GURGACZ,Rent,004.948.028-63,GILBERTO PISELO DO NASCIMENTO,002/18,Despesa com pagamento de aluguel de imóvel par...,6000.00


## Dicionario de nome de colunas <a name="diccolumns"></a>

ANO: Ano do reembolso. Data

MES: mês do ano corrente do reembolso. Data

SENADOR: Nome do Senador. 

TIPO_DESPESA: Tipo de despesas. Categórico
    - General expenses
    - Security

VALOR_REEMBOLSADO: Valor total do reembolso do gasto

Obs: Estão listadas as colunas utilizadas em nossa análise. As colunas não listadas aqui foram mantidas apenas para eventuais questões que possam ser levantadas no decorrer da análise.